In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2+3)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [9]:
SE18 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : alpha*c})

In [10]:
len(SE18.find_admissible_permutations())

1296

In [11]:
%time adm_SE18 = SE18.find_admissible_projectivities()

CPU times: user 1.77 s, sys: 133 ms, total: 1.91 s
Wall time: 13.8 s


In [12]:
len(adm_SE18)

648

In [13]:
%time simm_SE18 = SE18.find_simmetries(adm_SE18)

CPU times: user 1.39 s, sys: 65.7 ms, total: 1.46 s
Wall time: 1.48 s


In [14]:
len(simm_SE18)

648

In [15]:
G_SE18 = Group(simm_SE18)

In [16]:
%%time
for i in [0]:
    g1 = G_SE18[i]
    for g2 in G_SE18:
        G_SE18.is_in_group(g1*g2)

CPU times: user 1min 57s, sys: 717 ms, total: 1min 58s
Wall time: 1min 58s


In [17]:
#%time G_SE18.is_group()

In [18]:
G_SE18

Non abelian group of order 648

In [19]:
G_SE18.get_order()

648

In [20]:
G_SE18.is_abelian()

False

In [21]:
G_SE18.get_divisors_of_order()

[1, 2, 3, 4, 6, 8, 9, 12, 18, 24, 27, 36, 54, 72, 81, 108, 162, 216, 324, 648]

In [22]:
perms = []
for el in G_SE18:
    perms.append(Permutation(apply_proj_to_eck(el, SE18.eckardt_points)).to_cycles())

In [58]:
gp = PermutationGroup(perms)

In [59]:
gp.order()

648

In [60]:
gp.structure_description()

'(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2'

In [71]:
nor_subs = gp.normal_subgroups()

In [74]:
nor_sub = [el for el in nor_subs if el.order()==27][0]

In [75]:
gp.quotient(nor_sub).is_isomorphic(SymmetricGroup(4))

True

In [79]:
nor_sub.structure_description()

'C3 x C3 x C3'